In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [8]:
def naive_center(x):  # BROKEN! don't use!
    x = np.asarray(x)
    return x - np.mean(x)

In [9]:
import numpy as np
from patsy import dmatrix, build_design_matrices, incr_dbuilder
data = {"x": [1, 2, 3, 4]}

In [10]:
mat = dmatrix("naive_center(x)", data)
mat

DesignMatrix with shape (4, 2)
  Intercept  naive_center(x)
          1             -1.5
          1             -0.5
          1              0.5
          1              1.5
  Terms:
    'Intercept' (column 0)
    'naive_center(x)' (column 1)

In [11]:
new_data = {"x": [5, 6, 7, 8]}
build_design_matrices([mat.design_info], new_data)[0]

DesignMatrix with shape (4, 2)
  Intercept  naive_center(x)
          1             -1.5
          1             -0.5
          1              0.5
          1              1.5
  Terms:
    'Intercept' (column 0)
    'naive_center(x)' (column 1)

In [13]:
fixed_mat = dmatrix("center(x)", data)

fixed_mat

DesignMatrix with shape (4, 2)
  Intercept  center(x)
          1       -1.5
          1       -0.5
          1        0.5
          1        1.5
  Terms:
    'Intercept' (column 0)
    'center(x)' (column 1)

In [14]:
build_design_matrices([fixed_mat.design_info], new_data)[0]

DesignMatrix with shape (4, 2)
  Intercept  center(x)
          1        2.5
          1        3.5
          1        4.5
          1        5.5
  Terms:
    'Intercept' (column 0)
    'center(x)' (column 1)

In [15]:
data_chunked = [{"x": data["x"][:2]},
                {"x": data["x"][2:]}]

dinfo = incr_dbuilder("naive_center(x)", lambda: iter(data_chunked))
np.row_stack([build_design_matrices([dinfo], chunk)[0]
              for chunk in data_chunked])


array([[ 1. , -0.5],
       [ 1. ,  0.5],
       [ 1. , -0.5],
       [ 1. ,  0.5]])

In [16]:
dinfo = incr_dbuilder("center(x)", lambda: iter(data_chunked))
np.row_stack([build_design_matrices([dinfo], chunk)[0]
              for chunk in data_chunked])


array([[ 1. , -1.5],
       [ 1. , -0.5],
       [ 1. ,  0.5],
       [ 1. ,  1.5]])

In [17]:
build_design_matrices([dinfo], new_data)[0]

DesignMatrix with shape (4, 2)
  Intercept  center(x)
          1        2.5
          1        3.5
          1        4.5
          1        5.5
  Terms:
    'Intercept' (column 0)
    'center(x)' (column 1)

In [21]:
from patsy import stateful_transform

class MyExampleCenter(object):
    def __init__(self):
        self._total = 0
        self._count = 0
        self._mean = None

    def memorize_chunk(self, x):
        self._total += np.sum(x)
        self._count += len(x)

    def memorize_finish(self):
        self._mean = self._total * 1. / self._count

    def transform(self, x):
        return x - self._mean

my_example_center = stateful_transform(MyExampleCenter)
print(my_example_center(np.array([1, 2, 3])))

[-1.  0.  1.]


In [22]:
fixed_mat = dmatrix("standardize(x, center=False, rescale=True, ddof=1)", data)

fixed_mat

DesignMatrix with shape (4, 2)
  Intercept  standardize(x, center=False, rescale=True, ddof=1)
          1                                             0.77460
          1                                             1.54919
          1                                             2.32379
          1                                             3.09839
  Terms:
    'Intercept' (column 0)
    'standardize(x, center=False, rescale=True, ddof=1)' (column 1)